In [ ]:
from google.colab import drive
import os
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
root = "/gdrive/My Drive/CS470/cs470_music_generator/"

In [ ]:
!pip install pydub
!pip install gTTS
!pip install soundfile


In [ ]:
!sudo apt update
!sudo apt-get install timidity

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1,814 kB]
Get:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu

# Import packages

In [ ]:
import pickle
from gtts import gTTS
import wave
import contextlib
import librosa
import soundfile
from pydub import AudioSegment


# Merge songs

In [ ]:
#convert midi file to wav format 
#saved in the current directory as "out.wav"
song = root + 'music/song2.midi'
print(song)
!timidity "$song" -Ow -o out.wav

/gdrive/My Drive/CS470/cs470_music_generator/music/song2.midi
Playing /gdrive/My Drive/CS470/cs470_music_generator/music/song2.midi
MIDI file: /gdrive/My Drive/CS470/cs470_music_generator/music/song2.midi
Format: 1  Tracks: 1  Divisions: 1024
Sequence: Piano
Playing time: ~13 seconds
Notes cut: 0
Notes lost totally: 0


In [ ]:
with open(root+'data/last_generated.pkl','rb') as fb:
  lines = pickle.load(fb)
  fb.close()

lines = lines[0]

In [ ]:
lines

'goodness \n by \n big brother and the holding company \n \n you give me what you want \n by \n the animals \n \n paper planes \n by \n mia \n \n ill feel a whole lot \n by \n bill withers \n \n seven nation army \n by \n the white stripes \n \n you are the sunshine of my life \n by \n stevie wonder \n \n help me \n by \n joni mitchell \n \n call me \n by \n blondie \n \n whats so funny bout peace love and understanding \n by \n elvis costello and the attractions \n \n smoke stack lightning \n by \n howlin wolf \n \n summer babe \n by \n van morrison \n \n just like a woman \n by \n bob dylan \n \n sexual healing \n by \n marvin gaye \n \n only the lonely \n by \n roy orbison \n \n we gotta get out of this place \n by \n the animals \n \n paper planes \n by \n mia \n \n ill feel a whole lot better \n by \n the byrds \n \n everyday \n by \n buddy holly \n \n i got a good'

In [ ]:
lyrics = [line.replace('\n', '.') for line in lines]
lyrics = ''.join(lyrics)
# print(lyrics)

lyrics_tts = gTTS(text = lyrics, lang = 'en')#, slow = 'False')
lyrics_tts.save("lyrics.wav")

In [ ]:
x,_ = librosa.load('lyrics.wav', sr=16000)
soundfile.write('tmp.wav', x, 16000)
fopen = wave.open('tmp.wav','r')
with contextlib.closing(fopen) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration_lyrics = frames / float(rate) #in seconds
    print(duration_lyrics)

60.12


In [ ]:
fopen = wave.open('out.wav','r')
with contextlib.closing(fopen) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration_midi = frames / float(rate) #in seconds
    print(duration_midi)

11.75045351473923


In [ ]:
n = math.ceil(duration_lyrics/duration_midi) + 1
print(n)
chunk = AudioSegment.from_wav('out.wav')
output = chunk
for i in range(n):
  output += chunk
output += chunk
output.export("joinedFile.wav", format="wav")

4


<_io.BufferedRandom name='joinedFile.wav'>

In [ ]:
sound_text = AudioSegment.from_file("tmp.wav")
sound_melody = AudioSegment.from_file("joinedFile.wav")

# mix sound2 with sound1, starting at 5000ms into sound1)
print(duration_midi)
output = sound_melody.overlay(sound_text, position=duration_midi*1000)

# save the result
output.export("song.mp3", format="mp3")

11.75045351473923


<_io.BufferedRandom name='song.mp3'>